In [1]:
# author: Mathieu Hénault
from Bio import SeqIO
from Bio import SeqRecord as SeqRecord
import pandas as pd
import numpy as np
import itertools
from matplotlib import pyplot as plt

# Analysis of the alignments of draft assemblies using mummer

In [2]:
DELTA = {}
ASSEMBLIES = {}
tig_order = {}
cum_len = {}
for s in ['Jean-Talon_reordered.fasta','S288c.genome.fa','barcode11.cns.fa', 'Jean-Talon.unitigs.fasta', 'Jean-Talon.contigs.fasta']:
    ASSEMBLIES[s] = {}
    tig_order[s] = []
    cl = []
    with open(f'/Volumes/MacintoshHD/Dropbox/Jean_Talon/assemblies/{s}') as fi:
        for seq in SeqIO.parse(fi, 'fasta'):
            ASSEMBLIES[s][seq.id] = seq
            tig_order[s].append(seq.id)
            cl.append(len(seq.seq))
    
    cl = pd.Series([0]+list(np.cumsum(cl)[:-1]), index=tig_order[s])
    cum_len[s] = cl

    if s!='Jean-Talon_reordered.fasta':
        delta = pd.read_csv(f'/Volumes/MacintoshHD/Dropbox/Jean_Talon/mummer/{s}.coords', sep='\t', skiprows=range(4), index_col=None, header=None)
        delta = delta.loc[(delta[4]>=1e4) & (delta[5]>=1e4)].astype({1:int,2:int,2:int,2:int}).sort_values(by=4)
        
        delta['s1'] = delta[0]+cum_len['Jean-Talon_reordered.fasta'].loc[delta[7].values].values
        delta['e1'] = delta[1]+cum_len['Jean-Talon_reordered.fasta'].loc[delta[7].values].values
        delta['s2'] = delta[2]+cum_len[s].loc[delta[8].values].values
        delta['e2'] = delta[3]+cum_len[s].loc[delta[8].values].values
        
        DELTA[s] = delta        

In [3]:
fig = plt.figure(figsize=[32,8])
gs = plt.GridSpec(ncols=4, nrows=1, wspace=0.3, left=0.03, bottom=0.08, right=0.96, top=0.9)

assembly_alias = {'Jean-Talon_reordered.fasta': 'Jean-Talon wtdbg2 polished',
                  'S288c.genome.fa': 'S288C',
                  'barcode11.cns.fa': 'Jean-Talon wtdbg2 draft (>8kb reads)', 
                  'Jean-Talon.unitigs.fasta': 'Jean-Talon Canu draft unitigs (>8kb reads)', 
                  'Jean-Talon.contigs.fasta': 'Jean-Talon Canu draft contigs (>8kb reads)'}
coord_translocation = np.mean(np.array([1.4e5, 1.47e5])+cum_len['Jean-Talon_reordered.fasta'].loc['ctg6_pilon'])

for ax_idx, s in enumerate(['S288c.genome.fa', 'barcode11.cns.fa', 'Jean-Talon.contigs.fasta', 'Jean-Talon.unitigs.fasta']):
    ax = fig.add_subplot(gs[ax_idx])
    delta = DELTA[s]
    for i in delta.index:
        s1, e1, s2, e2 = delta.loc[i, ['s1','e1','s2','e2']]
        c = str(1-delta.loc[i,6]/100)
        ax.plot([s1,e1], [s2,e2], color=c, lw=1)
    for i in cum_len[s]:
        ax.axhline(i, lw=0.5, ls=':', color='k')
    for i in cum_len['Jean-Talon_reordered.fasta']:
        ax.axvline(i, lw=0.5, ls=':', color='k')
    ax.margins(0)
    ax.axvline(coord_translocation, color='red', ls='-', lw=3, alpha=0.3)
    
    assembly_size = ax.axis()
    ax.set_xticks(np.arange(0, assembly_size[1], 1e6))
    ax.set_xticklabels(np.arange(0, assembly_size[1]*1e-6, 1).astype(int))
    ax.set_yticks(np.arange(0, assembly_size[3], 1e6))
    ax.set_yticklabels(np.arange(0, assembly_size[3]*1e-6, 1).astype(int))
    
    #if s=='S288c.genome.fa':
    for tig, df in delta.groupby(8):
        if len(ASSEMBLIES[s][tig].seq)>2e5:
            ax.text(assembly_size[1]*1.02, np.mean([df['s2'].min(), df['e2'].max()]), s=tig, va='center', ha='left', size=7)
            
    for tig, df in delta.groupby(7):
        if len(ASSEMBLIES['Jean-Talon_reordered.fasta'][tig].seq)>2e5:
            ax.text(np.mean([df['s1'].min(), df['e1'].max()]), assembly_size[3]*1.02, s=tig, va='bottom', ha='center', rotation=90, size=7)
            
    ax.set_xlabel(assembly_alias['Jean-Talon_reordered.fasta'], size=14)
    ax.set_ylabel(assembly_alias[s], size=14)
    
plt.savefig('/Volumes/MacintoshHD/Dropbox/Jean_Talon/fig/FigS7.svg')
#plt.show()
plt.close()